In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
        
print(os.listdir("../input/new-york-city-taxi-fare-prediction"))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("../input/new-york-city-taxi-fare-prediction/train.csv", nrows = 1000000)
test = pd.read_csv("../input/new-york-city-taxi-fare-prediction/test.csv")

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
train.head(10)

In [ ]:
train.describe()

In [ ]:
#check for missing values in train data
train.isnull().sum().sort_values(ascending=False)

In [ ]:
#drop the missing values
train = train.drop(train[train.isnull().any(1)].index, axis = 0)

In [ ]:
train.shape

In [ ]:
#38 fields have negative fare_amount values.
from collections import Counter
Counter(train['fare_amount']<0)

In [ ]:
train = train.drop(train[train['fare_amount']<0].index, axis=0)
train.shape

In [ ]:
#no more negative values in the fare field
train['fare_amount'].describe()

In [ ]:
#highest fare is $500
train['fare_amount'].sort_values(ascending=False)

In [ ]:
train['passenger_count'].describe()

In [ ]:
#max is 208 passengers. Assuming that a bus is a 'taxi' in NYC, I don't think a bus can carry 208 passengers! Let' see the distribution of this field
#LOL! One field. this is DEFINITELY an outlier. Lets drop it 
train[train['passenger_count']>6]

In [ ]:
train = train.drop(train[train['passenger_count']==208].index, axis = 0)

In [ ]:
#much neater now! Max number of passengers are 6. Which makes sense is the cab is an SUV :)
train['passenger_count'].describe()

In [ ]:
#Next, let us explore the pickup latitude and longitudes
train['pickup_latitude'].describe()

In [ ]:
train.dtypes

データ前処理  
・済　欠損データ削除  
・済　料金　マイナスデータ削除  
・済　乗客6人以上を削除  
・(未)緯度の範囲を-90・90にする  
・(未)経度の範囲を-180・180にする  


In [ ]:
train['key'] = pd.to_datetime(train['key'])
train['pickup_datetime']  = pd.to_datetime(train['pickup_datetime'])

In [ ]:
#Convert for test data
test['key'] = pd.to_datetime(test['key'])
test['pickup_datetime']  = pd.to_datetime(test['pickup_datetime'])

In [ ]:
#check the dtypes after conversion
train.dtypes

In [ ]:
test.dtypes

In [ ]:
train.head()

In [ ]:
test.head()

**EDA**　　
移動距離を求め、追加する

In [ ]:
def haversine_distance(lat1, long1, lat2, long2):
    data = [train, test]
    for i in data:
        R = 6371  #radius of earth in kilometers
        #R = 3959 #radius of earth in miles
        phi1 = np.radians(i[lat1])
        phi2 = np.radians(i[lat2])
    
        delta_phi = np.radians(i[lat2]-i[lat1])
        delta_lambda = np.radians(i[long2]-i[long1])
    
        #a = sin²((φB - φA)/2) + cos φA . cos φB . sin²((λB - λA)/2)
        a = np.sin(delta_phi / 2.0) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2.0) ** 2
    
        #c = 2 * atan2( √a, √(1−a) )
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    
        #d = R*c
        d = (R * c) #in kilometers
        i['H_Distance'] = d
    return d

In [ ]:
haversine_distance('pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude')

In [ ]:
train['H_Distance'].head(10)

In [ ]:
test['H_Distance'].head(10)

In [ ]:
train.head(10)

In [ ]:
test.head(10)

**列の作成**  
年
月
日にち
時間
曜日

In [ ]:
data = [train,test]
for i in data:
    i['Year'] = i['pickup_datetime'].dt.year
    i['Month'] = i['pickup_datetime'].dt.month
    i['Date'] = i['pickup_datetime'].dt.day
    i['Day of Week'] = i['pickup_datetime'].dt.dayofweek
    i['Hour'] = i['pickup_datetime'].dt.hour

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.shape